In [1]:
import pandas as pd
import numpy as np

In [2]:
df_resale = pd.read_csv('../data/cleaned/resale_price_cleaned.csv')

### Add inflation-adjusted resale price
Real Price = Nominal Price * (CPI Base Year/ CPI Curret Month)

In [3]:
df_deflated_resale = df_resale.copy()
cpi_base = 100
df_deflated_resale["deflated_resale_price"] = df_deflated_resale["resale_price"] * (100/ df_deflated_resale["CPI (base 2024-12)"])

### Drop correlated variables

In [4]:
df_deflated_resale.drop(columns = ['resale_price', 'flat_type', 'building_age_2025', 'total_unemployment_rate', 'flat_model',
                                   'CPI (base 2024-12)'], inplace = True)


df_deflated_resale.dropna(inplace = True)

In [9]:
import os

current_directory = os.getcwd()
print(current_directory)

/Users/judy/Desktop/DSE3101-Project/DSE3101-HDB-Decode/code


### Add ethnicity and religion columns 

In [5]:
# from another dataset
df_2 = pd.read_csv("../data/cleaned/resale_price_normalized_demo.csv")

# drop NoReligion and Others
df_2 = df_2[['town','Buddhism','Taoism1','Islam','Hinduism','Sikhism',
            'Christianity_Catholic','Christianity_OtherChristians','OtherReligions','Chinese', 'Malays','Indians']]

df_2_selected = df_2[['town','Buddhism']]

In [7]:
df_deflated_resale_sorted = df_deflated_resale.sort_values('town').reset_index(drop=True)
df_2_sorted = df_2.sort_values('town').reset_index(drop=True)
(df_deflated_resale_sorted['town'] == df_2_sorted['town']).all()

True

In [8]:
columns_to_add = ['Buddhism','Taoism1','Islam','Hinduism','Sikhism',
                  'Christianity_Catholic','Christianity_OtherChristians','OtherReligions',
                  'Chinese', 'Malays','Indians']

for col in columns_to_add:
    df_deflated_resale_sorted[col] = df_2_sorted[col]

Merge takes up too much RAM memory...

In [ ]:
df_deflated_resale_small = df_deflated_resale.head(2000)
df_2_small = df_2.head(2000)
df_com = pd.merge(df_deflated_resale_small, df_2_small, on = 'town', how = 'left')

In [ ]:
df_com = pd.merge(df_deflated_resale, df_2_selected, on = 'town', how = 'left')

: 

In [9]:
df_deflated_resale_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196986 entries, 0 to 196985
Data columns (total 31 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         196986 non-null  object 
 1   town                          196986 non-null  object 
 2   storey_range                  196986 non-null  int64  
 3   floor_area_sqm                196986 non-null  float64
 4   remaining_lease               196986 non-null  float64
 5   lat                           196986 non-null  float64
 6   lon                           196986 non-null  float64
 7   nearest_mrt_distance          196986 non-null  float64
 8   nearest_bus_distance          196986 non-null  float64
 9   education_score               196986 non-null  float64
 10  shopping_score                196986 non-null  float64
 11  food_score                    196986 non-null  float64
 12  recreation_score              196986 non-nul

### Standardization (z-score)

In [10]:
df_resale_standardized = df_deflated_resale_sorted.copy()
def standardized(col):
    return (col - col.mean()) / col.std()

columns_to_standardized = df_deflated_resale.select_dtypes(include=['float64', 'int64']).columns.difference(['lat', 'lon'])
df_resale_standardized[columns_to_standardized] = df_deflated_resale[columns_to_standardized].apply(standardized)
df_resale_standardized.head()


,month,town,storey_range,floor_area_sqm,remaining_lease,lat,lon,nearest_mrt_distance,nearest_bus_distance,education_score,...,Taoism1,Islam,Hinduism,Sikhism,Christianity_Catholic,Christianity_OtherChristians,OtherReligions,Chinese,Malays,Indians
0,2024-01,ANG MO KIO,0.883747,-1.581737,-1.166418,1.370561,103.854823,-0.097301,-0.247573,-1.850318,...,0.120314,0.076259,0.046381,0.002381,0.084552,0.113769,0.00149,0.739422,0.142192,0.089282
1,2022-05,ANG MO KIO,-0.127351,-1.581737,-1.177795,1.362758,103.858015,-0.097301,-0.247573,-1.850318,...,0.120314,0.076259,0.046381,0.002381,0.084552,0.113769,0.00149,0.739422,0.142192,0.089282
2,2022-07,ANG MO KIO,0.378198,-1.581737,-1.225435,1.362758,103.858015,-0.097301,-0.247573,-1.850318,...,0.120314,0.076259,0.046381,0.002381,0.084552,0.113769,0.00149,0.739422,0.142192,0.089282
3,2022-08,ANG MO KIO,-1.138449,-1.581737,-1.248899,1.362758,103.858015,-0.097301,-0.247573,-1.850318,...,0.120314,0.076259,0.046381,0.002381,0.084552,0.113769,0.00149,0.739422,0.142192,0.089282
4,2023-01,ANG MO KIO,-0.632900,-1.581737,-1.248899,1.362758,103.858015,-0.097301,-0.247573,-1.850318,...,0.120314,0.076259,0.046381,0.002381,0.084552,0.113769,0.00149,0.739422,0.142192,0.089282


### One-Hot encoding

In [11]:
# drop the first value to avoid dummy variable trap
df_encoded = pd.get_dummies(df_resale_standardized, columns=['town'], drop_first=True)

In [12]:
df_encoded.to_csv('../data/cleaned/standardized_encoded_delfated_resale_price.csv', index=False)